
# 1. Inizializzazione e import delle librerie

*   Collegamento a Google Drive e import delle librerie necessarie

In [ ]:
# Before running the script, reset the runtime to factory reset (Runtime -> Factory Reset Runtime)
# and then change runtime type to GPU (Runtime -> Change runtime type)

# Install libriary dependencies for running deep learning
!pip install tensorflow #==2.9.0          #2.1.0
!pip install keras #==2.3.1
# !pip install segmentation_models #==1.0.1
!pip install h5py #==2.10.0 

# Per le reti 3D
# !pip install classification-models-3D
# !pip install segmentation-models-3D

!pip3 install SimpleITK
!pip install numpyencoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48.4 MB 1.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path

import os
from tabulate import tabulate
import numpy as np
import nibabel as nib
from tqdm import tqdm

import matplotlib.pyplot as plt
import skimage
from skimage.transform import resize
from skimage.util import *
from skimage import measure
import skimage.metrics as ski_metrics
from scipy import ndimage
from scipy.ndimage import zoom
import plotly.express as px

import json
import time
import math
import copy

import SimpleITK as sitk
from numpyencoder import NumpyEncoder

# Import delle librerie aggiuntive per la lettura e visualizzazione
current_dir = '/content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge'
os.chdir(current_dir)
from utils import load_volume
from utils import load_segmentation
from visualize import visualize

# import import_ipynb
# from utilities import *

from keras.models import load_model

Mounted at /content/drive


In [ ]:
# !unzip -u '/content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge/test.zip' -d '//content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge/'

In [ ]:
path_train = current_dir+'/'+'training'
train_vol = sorted(os.listdir(path_train)) 

path_val = current_dir+'/'+'validation'
val_vol = sorted(os.listdir(path_val)) 

path_test = current_dir+'/'+'test'
test_vol = sorted(os.listdir(path_test)) 

In [ ]:
# # Indirizzamento (o creazione) cartelle che contengono il TRAINING SET, il VALIDATION SET o il TEST SET
# path_data = create_path(current_dir, 'data')                               # CAMBIARE NOME DATASET A SECONDA DELLA PROVA
# # path_train_vol = create_path(path_data,'train_volumes')
# # path_train_mask = create_path(path_data, 'train_masks')
# # path_val_vol = create_path(path_data, 'val_volumes')
# # path_val_mask = create_path(path_data, 'val_masks')

# path_test_vol = create_path(path_data, 'test_volumes')

In [ ]:
# file_path = '/content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge/train_info_128'
# fp= open(file_path)
# volumes_train_info = json.loads(fp.read())

# file_path = '/content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge/val_info_128'
# fp= open(file_path)
# volumes_val_info = json.loads(fp.read())

## Funzioni

In [ ]:
def create_path(gen_path, folder_name):
  # Funzione che riceve in input il nome di una cartella da creare e restituisce il percorso 
  # appena generato; se la cartella indicata è già presente all'interno del drive viene 
  # restituito il percorso (già esistente) relativo a questa
  # args:   - gen_path: percorso 'genitore' all'interno del quale viene creata la cartella folder_name
  #         - folder_name: nome della cartella da creare (o da indirizzare, se già presente nel drive)
  # output: - path: percorso relativo alla cartella folder_name
  path = os.path.join(gen_path, folder_name)
  if not os.path.exists(path):
      os.mkdir(path)
  return path

In [ ]:
def dict_fill(data_itk, vol_info, n, data_itk_prep = None):
# def dict_fill(data_itk, vol_info):
    # Questa funzione riempie un dizionario che conterrà, per ciascun caso, le informazioni relative a:
    # numero di slice del volume, risoluzione lungo l'asse x, risoluzione lungo l'asse y, spessore della slice
    # NB: il dizionario deve essere indicizzato nel seguente modo -> volumes_train_info['key'][case_number]
    #     dove case_number è un numero compreso nell'intervallo 0-100 per il training set o 0-49 per il validation set
    # args:   - data_itk: volume da cui estrarre le informazioni
    #         - vol_info: dizionario vuoto
    # output: - vol_info: dizionario riempito con le informazioni
    n_key = "{}".format(n)

    if data_itk_prep:
      vol_info[n_key]["spacing_2D"] = list(data_itk.GetSpacing())     # mm/pixel 
      vol_info[n_key]["obtained_size_2D"] = list(data_itk.GetSize())

    else:
      vol_info[n_key] = {}

      n_slices,height,width = np.array(data_itk.GetSize())
      vol_info[n_key]["n_slices"] = int(n_slices)                                        
      vol_info[n_key]["original_spacing"] = list(data_itk.GetSpacing())    # mm/pixel 
      vol_info[n_key]["original_size"] = list((n_slices,height,width))

    return vol_info

In [ ]:
def resample_image(image, out_spacing=(1.0, 1.0, 1.0), out_size=None, is_label=False, pad_value=0, is_prediction= False, is_3D = False):
    """Resamples an image to given element spacing and output size."""

    original_spacing = np.array(image.GetSpacing())
    original_size = np.array(image.GetSize())

    if len(out_spacing)==2:
      out_spacing = (original_spacing[0], out_spacing[0], out_spacing[1])

    if out_size is None:
        out_size = np.round(np.array(original_size * original_spacing / np.array(out_spacing))).astype(int)
    else:
      if len(out_size)==2:
        out_size = np.array(out_size)
        out_size_temp = np.round(np.array(original_size * original_spacing / np.array(out_spacing))).astype(int)
        out_size = np.insert(out_size,0,out_size_temp[0])
      elif len(out_size)==3:
        out_size = np.array(out_size)
    
    original_direction = np.array(image.GetDirection()).reshape(len(original_spacing),-1)
    original_center = (np.array(original_size, dtype=float) - 1.0) / 2.0 * original_spacing
    out_center = (np.array(out_size, dtype=float) - 1.0) / 2.0 * np.array(out_spacing)

    original_center = np.matmul(original_direction, original_center)
    out_center = np.matmul(original_direction, out_center)
    out_origin = np.array(image.GetOrigin()) + (original_center - out_center)

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size.tolist())
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(out_origin.tolist())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(pad_value)

    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkLinear)

    if is_prediction:
        return resample.Execute(sitk.Cast(image, sitk.sitkFloat32))
    elif is_3D:
        return resample.Execute(image)
    else:
        return resample.Execute(sitk.Cast(image, sitk.sitkUInt8))  

In [ ]:
# Funzioni utilizzate per passare dal formato di importazione delle SimpleITK 
# all'orientamento utilizzato per le matrici e vice versa

def sitk_image_to_data(image):
    data = sitk.GetArrayFromImage(image)
    if len(data.shape) == 3:
      data = np.rot90(data, 1, axes=(0, 2))     # rotazione con cui è stata allenata la rete
    return data

def data_to_sitk_image(data, spacing=(1., 1., 1.),is_2D = False):
    if len(data.shape) == 3:
      data = np.rot90(data, -1, axes=(0, 2))     # rotazione con cui è stata allenata la rete
    image = sitk.GetImageFromArray(data)
    image.SetSpacing(np.asarray(spacing, dtype= float))
    return image

In [ ]:
# Funzione utilizzata per ricampionare le segmentazioni automatiche alla risoluzione originale
def to_original_spacing(data, volumes_train_info, n):     
    original_spacing = volumes_train_info["{}".format(n)]['original_spacing']
    original_size = volumes_train_info["{}".format(n)]['original_size']

    data_itk = data_to_sitk_image(data, spacing = volumes_train_info["{}".format(n)]['spacing_2D'])
    data_itk = resample_image(data_itk, out_spacing = original_spacing, out_size = original_size,  is_label = True, is_prediction=True)
    data_out = sitk_image_to_data(data_itk)
    return data_out

In [ ]:
def hu_clip(volume, hu_min, hu_max):
    # Le intensità dell'immagine vengono fissate tra hu_min e hu_max
    volume = np.clip(volume, hu_min, hu_max)
    return volume

def scale(volume):
    # Le intensità dell'immagine vengono normalizzate tra 0 e 1...
    mxval = np.max(volume)
    mnval = np.min(volume)
    im_volume = (volume - mnval)/(mxval - mnval)
    # ...e scalate tra 0 e 255
    im_volume = 255*im_volume
    im_volume = im_volume.astype(np.uint8)
    return im_volume

In [ ]:
def display2(img1, img2, title1='Image 1', title2='Image 2', color_bar = True): 
  fig = plt.figure(figsize=(12, 6))
  ax1 = plt.subplot(1, 3, 1)
  z1_plot = ax1.imshow(img1, cmap=plt.cm.gray)
  ax1.set_title(title1)
  if color_bar:
   fig.colorbar(z1_plot, ax=ax1)

  ax2 = plt.subplot(1, 3, 2)
  z2_plot = ax2.imshow(img2, cmap=plt.cm.gray)
  ax2.set_title(title2)
  if color_bar:
    fig.colorbar(z2_plot, ax=ax2)

In [ ]:
def display_slicer(mask_3d):
  print(f'Slice dimension...: ', mask_3d.shape[1], 'x', mask_3d.shape[2],'\nSlice number......: ', mask_3d.shape[0])
  fig = px.imshow(mask_3d[:,:,:], animation_frame=0, binary_string=True, labels=dict(animation_frame='slice'))
  fig.show()

## Caricamento dati e setting parametri

*   Estrazione del test set dalla cartelle compressa nella cartelle creata nel drive



In [ ]:
# !unzip -u '/content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge/training.zip' -d '//content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge/'
# !unzip -u '/content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge/validation.zip' -d '//content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge/'
# !unzip -u '/content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge/test.zip' -d '//content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge/'

* Load dei modelli allenati

In [ ]:
name_model_2D = '2D_Unet.h5'
model_2D = load_model(current_dir+'/'+name_model_2D, compile= False)

name_model_3D = '3D_Unet.h5'
model_3D = load_model(current_dir+'/'+name_model_3D, compile= False)

* Setting parametri


In [ ]:
IMG_WIDTH = 384    
IMG_HEIGHT = 384
NUM_CLASSES_2D = 2
hu_min = -100
hu_max = 400

* Creazione dei dizionari

In [ ]:
volumes_train_info = {}
volumes_val_info = {}
volumes_test_info = {}

# 3. Preprocess 2D

Step da seguire:


1.   Clippare le intensità delle immagini all'interno della finestra definita tramite l'osservazione dell'istogramma di alcuni casi; l'intervo selezionato è [-100, 400] HU
2.   Normalizzare le intensità delle immagini tra 0 e 1 e scaling tra 0 e 255, al fine di renderle confrontabili anche se acquisite con dispositivi diversi 
3.   Ricampionamento delle sul piano xy ad una dimensione media pari a circa [1,1] mm/px
4.   Crop o Zero-padding per adattare le dimensioni del'immagine ricampionata alla input shape della rete 2D
5.   I volumi vengono convertiti in formato RGB, le maschere vengono binarizzate per ottenere 2 classi (0: sfondo, 1: rene e tumore)



In [ ]:
def preprocessing_2D(data_itk, n, volumes_info, is_label= False):
  if is_label:
    # Preprocessing per la rete 2D: ricampionamento sul piano xy a (1,1) mm/px
    data_out_itk = resample_image(data_itk, out_spacing=(1.0, 1.0),  out_size=(384,384), is_label=True)  
    data_out = sitk_image_to_data(data_out_itk)
    # Binarizzazione della maschera (classe 0: sfondo, classe 1: rene e tumore)
    data_out[data_out>=1] = 1 
  else:
    # Riempio il dizionario con le informazioni relative al volume 
    volumes_info = dict_fill(data_itk, volumes_info, n)
    # Preprocessing per la rete 2D: HU clipping, min-max Scaling, rescaling su 0-255, ricampionamento sul piano xy a (1,1) mm/px
    original_spacing = data_itk.GetSpacing()
    data_np = sitk_image_to_data(data_itk)
    data_np = hu_clip(data_np, -100, 400)
    data_np = scale(data_np)
    data_out_itk = data_to_sitk_image(data_np, spacing=original_spacing,is_2D = True)
    data_out_itk = resample_image(data_out_itk, out_spacing=(1.0, 1.0), out_size=(384,384)) 
    # Riempio il dizionario con le informazioni relative al preprocessing effettuato sul volume
    volumes_info = dict_fill(data_out_itk, volumes_info,n, data_itk_prep = True)  
    # Conversione a RGB        
    data_out = sitk_image_to_data(data_out_itk)  
  return data_out

# 4. Predict maschere 2D

In [ ]:
def predict_slice(vol_slice, volumes_info,n):
    softmax = model_2D.predict(np.expand_dims(vol_slice, axis=0))
    softmax = np.reshape(softmax,(IMG_HEIGHT,IMG_WIDTH,NUM_CLASSES_2D))
    prediction_slice = softmax[:,:,1]   
    
    prediction_slice[prediction_slice<=0.5]=0
    prediction_slice[prediction_slice>0.5]=1
    prediction_slice = prediction_slice.astype(np.uint8)
    return prediction_slice

In [ ]:
def predict_2D(volume, volumes_info, n):
  
  # predizione
  prediction_2D = np.zeros(volume.shape[0:3], dtype=np.uint8)
  for sl in range(0,volume.shape[0]):
    prediction_sl = predict_slice(volume[sl], volumes_info, n)
    prediction_2D[sl,:,:] = prediction_sl

  return prediction_2D

# 5. Preprocess 3D


Gli step seguiti per la creazione delle ROI vengono di seguito riportati:
* si individuano le due regioni presenti nel volume e si sceglie come punto di taglio il punto medio della distanza tra le stesse;
* una volta individuato il punto di taglio si suddivide il volume in due sub-volumi;
* si identificano i centroidi delle due aree;
* si individuano le bounding box attorno alle due aree;
* il centro delle ROI viene fatto coincidere con il centroide calcolato per ciascun complesso rene-tumore
* si effettua il crop delle ROI desiderate

In [ ]:
def cut_selection(mask_auto, n, volumes_info): 
  # - la funzione label del modulo measure esegue la connected component analysis;
  #   dandole in input un volume ci restituisce una maschera delle stesse dimensioni 
  #   di questo, al cui interno viene associata una label diversa ad ogni regione individuata  
  #   NB: con connectivity=3 diciamo di essere interessati all'individuazione di 
  #       componenti connessi nelle 3 dimensioni (quindi volumi)
  # - la funzione regionprops restituisce le proprietà di ciascuna delle regioni individuate
  aree = []             # vettore delle aree delle regioni - ci sarà utile per individuare le due aree maggiori
  labels = []           # vettore che conterrà le label che sono state associate a ciascuna regione individuata
  centroids = []        # vettore dei centroidi delle regioni; questa informazione ci servirà alla fine di tutto, quando dovremo ricostruire le dimensioni iniziali avremo bisogno di sapere dove 'ancorare' le maschere automatiche fornite in output dalla 3D
  bboxes = []
  mask_labels = measure.label(mask_auto, connectivity = 3)
  regions = measure.regionprops(label_image=mask_labels)
  for region in regions:
    aree.append(region.area)
    labels.append(region.label)
    centroids.append(region.centroid)
    bboxes.append(region.bbox)

  # Hp: le regioni di interesse sono quelle con estensione maggiore
  # nota: potremmo perdere qualche tumore separato dal rene
  area1 = max(aree)
  ind1 = aree.index(area1)
  label1 = labels[ind1]
  centroid1 = np.round(np.array(centroids[ind1]))

  bbox1 = bboxes[ind1]
  volumes_info['{}'.format(n)]['centroid_1_out'] = list(centroid1)     
  aree.pop(ind1)
  labels.pop(ind1)
  centroids.pop(ind1)
  
  if (len(aree)!=0) and (max(aree)>area1/25):    # condizione per considerare solo aree 'significative'  
    area2 = max(aree)
    ind2 = aree.index(area2)
    label2 = labels[ind2]
    centroid2 = np.round(np.array(centroids[ind2]))
    if centroid2[2] in range(bbox1[2],bbox1[5]):
      # Non considerare più questa regione
      aree.pop(ind2)
      labels.pop(ind2)
      centroids.pop(ind2)
      # cerca la successiva 
      area2 = max(aree)
      ind2 = aree.index(area2)
      label2 = labels[ind2]
      centroid2 = np.round(np.array(centroids[ind2]))

    volumes_info['{}'.format(n)]['centroid_2_out'] = list(centroid2)      
    # selezione della coordinata (x) di 'taglio'
    for region in regions:
      if region.label == label1:
        x1_min = region.bbox[2]
        x1_max = region.bbox[5]
        # print('bbox1: ', region.bbox)
      elif region.label == label2:
        x2_min = region.bbox[2]
        x2_max = region.bbox[5]
        # print('bbox2: ', region.bbox)
    if x1_max > x2_max:    
      # la regione 1 si trova a destra
      if x1_min < x2_max:  # se le due regioni sono parzialmente sovrapposte...
        #...scegliamo come punto di taglio il punto medio della sovrapposizione
        cut = x2_max + ((x2_max-x1_min)/2)    
      else:
        #...altrimenti il punto medio dello spazio "vuoto" tra le due regioni
        cut = x2_max + (x1_min-x2_max)/2      
    else:             
      # la regione 2 si trova a destra
      if x2_min < x1_max:
        cut = x1_max + ((x1_max-x2_min)/2)
      else:
        cut = x1_max + (x2_min-x1_max)/2
  else:
    # se è presente un solo grande componente connesso (e.g. caso 5), allora non viene effettuato alcun taglio 
    cut = 0      

  volumes_info['{}'.format(n)]['cut'] = int(cut)
  return int(cut), volumes_info

def volume_separation(volume, cut):
  volume1 = volume[:,:,:cut]
  volume2 = volume[:,:,cut:]
  return volume1, volume2

In [ ]:
def get_centroid(subvolume):
  aree = []             # vettore delle aree delle regioni - ci sarà utile per individuare le due aree maggiori
  centroids = []        # vettore dei centroidi delle regioni
  
  mask_labels = measure.label(subvolume, connectivity = 3)
  regions = measure.regionprops(label_image=mask_labels)
  for region in regions:
    aree.append(region.area)
    centroids.append(region.centroid)
  area1 = max(aree)
  ind1 = aree.index(area1)
  centroid = np.round(np.array(centroids[ind1])).astype(int)
  return centroid

In [ ]:
def get_bbox(segmentation):
    aree = []
    mask_labels = measure.label(segmentation, connectivity = 3)
    regions = measure.regionprops(label_image=mask_labels)
    for region in regions:
      aree.append(region.area)
    ind1 = aree.index(max(aree))
    region = regions[ind1]
    return region.bbox

def get_bbox_from_mask(mask, outside_value=0):
    mask_voxel_coords = np.where(mask != outside_value)
    minzidx = int(np.min(mask_voxel_coords[0]))
    maxzidx = int(np.max(mask_voxel_coords[0])) + 1
    minxidx = int(np.min(mask_voxel_coords[1]))
    maxxidx = int(np.max(mask_voxel_coords[1])) + 1
    minyidx = int(np.min(mask_voxel_coords[2]))
    maxyidx = int(np.max(mask_voxel_coords[2])) + 1
    return [[minzidx, maxzidx], [minxidx, maxxidx], [minyidx, maxyidx]]

def check_bbox(vol_auto_half, n, VOI_shape):
  bbox = get_bbox(vol_auto_half)
  d = bbox[3]-bbox[0]
  h = bbox[4]-bbox[1]
  w = bbox[5]-bbox[2] 

  # Scommetare sotto solo nella fase di preparazione all'allenamento della 3D
  # if d>VOI_shape[0] and (h>VOI_shape[1] or w>VOI_shape[2]) :
  #   print(f'\nCaso {n}: la bounding-box ha dimensioni superiori alla VOI - ({d},{h},{w})')
  # elif d>VOI_shape[0]: 
  #   print(f'\nCaso {n}: la bounding-box ha dimensioni superiori alla VOI - ({d},{h},{w})')
  # elif (h>VOI_shape[1] or w>VOI_shape[2]):
  #   print(f'\nCaso {n}: la bounding-box ha dimensioni superiori alla VOI - ({d},{h},{w})')
  return bbox

In [ ]:
def get_ROI(volume, centroid, ROI_shape):
    # ROI_shape deve essere un tuple contenente le dimensioni della ROI (e.g. (96,128,128))
    VD,VH,VW = volume.shape
    RD,RH,RW = ROI_shape
    cz,cy,cx=centroid

    if cx-RW//2 < 0:
      w_min = 0
      ROI_w_min = RW//2 - cx
    else:
      w_min = cx-RW//2
      ROI_w_min = 0
    if cx+RW//2 > VW-1:
      w_max = VW-1
      ROI_w_max = (RW//2)+VW-1-cx
    else:
      w_max = cx+RW//2
      ROI_w_max = RW 

    if cy-RH//2 < 0:
      h_min = 0
      ROI_h_min = RH//2 - cy
    else:
      h_min = cy-RH//2
      ROI_h_min = 0
    if cy+RH//2 > VH-1:
      h_max = VH-1
      ROI_h_max = (RH//2)+VH-1-cy
    else:
      h_max = cy+RH//2
      ROI_h_max = RH 

    if cz-RD//2 < 0:
      d_min = 0
      ROI_d_min = RD//2 - cz
    else:
      d_min = cz-RD//2
      ROI_d_min = 0
    if cz+RD//2 > VD-1:
      d_max = VD-1
      ROI_d_max = (RD//2)+VD-1-cz
    else:
      d_max = cz+RD//2
      ROI_d_max = RD      

    ROI = np.zeros(ROI_shape)
    ROI[ROI_d_min:ROI_d_max,ROI_h_min:ROI_h_max,ROI_w_min:ROI_w_max] = volume[d_min:d_max,h_min:h_max,w_min:w_max]
      
    return ROI

In [ ]:
def alignment(SEG, MASK, centroid):
  # descrizione: Questa funzione serve a posizionare il risultato della rete 3D all'interno 
  #              di un volume di segmentazione che servirà a ricostruire il volume di partenza 
  # args:        - SEG: 3d array inizializzato a zero al cui interno verrà posizionata MASK; []
  #              - MASK: maschera automatica output della 3D, viene posizionata all'interno di SEG facendo coincidere 
  #                      il suo centroide con il centroide originale del volume a cui si riferisce
  #              - centroid: centroide nel SR di SEG, quindi riferito al volume ricostruito nel postprocess della 2D
  # funzioni:    get_centroid(), get_bbox()

  # coordinate del centroide di riferimento (SR di SEG)
  z_cs, y_cs, x_cs = centroid
  # print("centroide salvato: {}".format(centroid))
  z_cs = int(np.ceil(z_cs))
  y_cs = int(np.ceil(y_cs))
  x_cs = int(np.ceil(x_cs))
  # estrazione del centroide dalla maschera (SR di MASK)
  MASK_bin = copy.deepcopy(MASK)
  MASK_bin[MASK_bin>1]=1

  if sum(sum(sum(MASK_bin))) != 0:
    C = get_centroid(MASK_bin)
    z_cm, y_cm, x_cm = C
    z_cm = int(np.ceil(z_cm))
    y_cm = int(np.ceil(y_cm))
    x_cm = int(np.ceil(x_cm))
    # estrazione delle coordinate della bounding box che racchiudono solo il volume individuato
    bbox = get_bbox(MASK_bin)
    z_min, y_min, x_min, z_max, y_max, x_max = bbox
    # calcolo degli spostamenti della bounding box rispetto al centroide della maschera automatica (C)
    dx_2 = x_max - x_cm
    dx_1 = x_cm - x_min
    dy_2 = y_max - y_cm
    dy_1 = y_cm - y_min
    dz_2 = z_max - z_cm
    dz_1 = z_cm - z_min
    
    # Per risolvere il problema velocemente porto i negativi a 0
    Z1 = z_cs-dz_1
    Z2 = z_cs+dz_2
    Y1 = y_cs-dy_1
    Y2 = y_cs+dy_2
    X1 = x_cs-dx_1
    X2 = x_cs+dx_2

    if Z1 < 0:
      z_min = z_min + abs(Z1)
      Z1 = np.clip(z_cs-dz_1, 0, 1000)
    elif Y1 < 0:
      y_min = y_min + abs(Y1)
      Y1 = np.clip(y_cs-dy_1, 0, 1000)
    elif X1 <0:
      x_min = x_min + abs(X1)
      X1 = np.clip(x_cs-dx_1, 0, 1000)
    if Z2>SEG.shape[0]:
      z_max = z_max - (Z2-SEG.shape[0])
    if Y2>SEG.shape[1]:
      y_max = y_max - (Y2-SEG.shape[1])
    if X2>SEG.shape[2]:
      x_max = x_max - (X2-SEG.shape[2])

    SEG[Z1:Z2, Y1:Y2, X1:X2] = MASK[z_min:z_max, y_min:y_max, x_min:x_max]

  return SEG

In [ ]:
# CONTROLLO - NUOVA da usare
def preprocess_3D(mask_auto, volume, n, volumes_info, is_val = False, is_test = False ):
  # volume_auto: segmentazione ottenuta dalla 2D (volume ricostruito)
  # volume: segmentazione manuale (per train e val), o volume (per train, val, test)
  
  out_spacing = (1.5, 1.5, 1.5)
  ROI_shape = (128,128,128)
  spacing_2D = volumes_info["{}".format(n)]["spacing_2D"]
  obtained_size_2D = volumes_info["{}".format(n)]["obtained_size_2D"]

  # converto maschera e volume a immagine ITK per ricampionare la risoluzione
  mask_auto_itk = data_to_sitk_image(mask_auto, spacing = spacing_2D)
  volume_itk = data_to_sitk_image(volume, spacing = spacing_2D)
  
  # Ricampiono alla risoluzione di input della 3D
  volume_itk = resample_image(volume_itk, out_spacing=out_spacing, out_size = (384,384), is_label=False, pad_value=0, is_3D = True)
  volume = sitk_image_to_data(volume_itk)
  mask_auto_itk = resample_image(mask_auto_itk, out_spacing=out_spacing, out_size = (384,384), is_label=True, pad_value=0)
  mask_auto = sitk_image_to_data(mask_auto_itk)
  volumes_info["{}".format(n)]["obtained_size_3D"] = mask_auto.shape

  # 1) Individuazione taglio e separazione volumi (individuazione centroidi di riferimento nella maschera che andrà riempita dopo la 3D; salvo tutte le info nel dizionario)
  cut, volumes_info = cut_selection(mask_auto, n, volumes_info)
  if cut == 0:
    # In questo caso è presente un'unica regione, non va effettuato alcun taglio
    bbox_1 = check_bbox(mask_auto, n, ROI_shape)
    volumes_info["{}".format(n)]["bbox_1"] = volumes_info["{}".format(n)]["bbox_2"] =  bbox_1
    centroid_1 = get_centroid(mask_auto) 
    volumes_info["{}".format(n)]["centroid_1_out"] = volumes_info["{}".format(n)]["centroid_2_out"] =  centroid_1
    # Allineamento dei centroidi e cropping della ROI desiderata 
    ROI1_vol = get_ROI(volume,centroid_1,ROI_shape).astype(np.uint8)
    ROI2_vol = copy.deepcopy(ROI1_vol).astype(np.uint8)
  else:
    vol1, vol2 = volume_separation(volume, cut)
    msk1, msk2 = volume_separation(mask_auto, cut)
    # 2) Individuazione bounding box attorno ai due complessi
    bbox_1 = check_bbox(msk1, n, ROI_shape)
    volumes_info["{}".format(n)]["bbox_1"] = bbox_1
    bbox_2 = check_bbox(msk2, n, ROI_shape)
    volumes_info["{}".format(n)]["bbox_2"] = bbox_2
    # 3) Individuo i centroidi dei due reni
    centroid_1 = get_centroid(msk1)
    centroid_2 = get_centroid(msk2)
    # 4) Allineamento dei centroidi e cropping della ROI desiderata 
    ROI1_vol = get_ROI(vol1,centroid_1,ROI_shape).astype(np.uint8)
    ROI2_vol = get_ROI(vol2,centroid_2,ROI_shape).astype(np.uint8)

  return ROI1_vol, ROI2_vol, volumes_info

# 6. Predict maschere 3D

* Predict 3D

In [ ]:
ROI_DEPTH, ROI_HEIGHT, ROI_WIDTH = [128,128,128]
N_CLASSES_3D = 3

In [ ]:
def predict3D(volume):
    softmax = model_3D.predict(np.expand_dims(volume, axis=0))
    softmax = np.reshape(softmax,(ROI_DEPTH, ROI_HEIGHT, ROI_WIDTH,N_CLASSES_3D))

    prediction = np.argmax(softmax,axis=-1)
    prediction = prediction.astype(np.uint8)
    return prediction 

* Ricostruzione Volume

In [ ]:
def postprocess_3D(MASK_AUTO_1, MASK_AUTO_2, n, volumes_info):
  n_key = "{}".format(n)                                    # conversione per rendere compatibile l'indice numerico con la key del dizionario
  obtained_size_3D = volumes_info[n_key]["obtained_size_3D"]
  original_spacing = volumes_info[n_key]["original_spacing"]
  original_size = volumes_info[n_key]["original_size"]
  spacing_3D = [1.5,1.5,1.5]
  SEG_AUTO = np.zeros((obtained_size_3D),dtype=np.float32)                   # inizializzazione di una matrice di zeri che conterrà le due ROI output della rete 3D  

  centroid_1 = volumes_info[n_key]["centroid_1_out"]   
  centroid_2 = volumes_info[n_key]["centroid_2_out"]

  x_c1 = centroid_1[2]
  x_c2 = centroid_2[2]
  if x_c2 < x_c1:
    centroid_2 = copy.deepcopy(volumes_info["{}".format(n)]["centroid_1_out"])
    centroid_1 = copy.deepcopy(volumes_info["{}".format(n)]["centroid_2_out"])

  if n == 5:
    SEG_AUTO = alignment(SEG_AUTO, MASK_AUTO_1, centroid_1)
  else:
    # ciascuna ROI viene posizionata all'interno del 3d ndarray inizializzato a 0, prestando attenzione a far coincidere i centroidi
    SEG_AUTO = alignment(SEG_AUTO, MASK_AUTO_1, centroid_1) 
    SEG_AUTO = alignment(SEG_AUTO, MASK_AUTO_2, centroid_2)  

  # converto maschera e volume a immagine ITK per ricampionare la risoluzione
  SEG_AUTO = data_to_sitk_image(SEG_AUTO, spacing = spacing_3D)

  # Ricampiono alla risoluzione originale
  SEG_AUTO = resample_image(SEG_AUTO, out_spacing=original_spacing, out_size = original_size, is_label=True, pad_value=0, is_prediction= True)
  SEG_AUTO = sitk_image_to_data(SEG_AUTO)

  return SEG_AUTO.astype(np.uint8)

### Metriche

* Dice score

In [ ]:
def compute_dice_coefficient(mask_gt, mask_pred):
  volume_sum = mask_gt.sum() + mask_pred.sum()
  if volume_sum == 0:
    return np.NaN
  volume_intersect = (mask_gt & mask_pred).sum()
  return 2*volume_intersect / volume_sum 

In [ ]:
def DSC(segmentation,prediction):
  prediction_kidney_tumor = np.array(prediction>=1)
  prediction_tumor = np.array(prediction==2)
  segmentation_kidney_tumor = np.array(segmentation>=1)
  segmentation_tumor = np.array(segmentation==2)
  dice_kidney_tumor = compute_dice_coefficient(segmentation_kidney_tumor, prediction_kidney_tumor)
  dice_tumor = compute_dice_coefficient(segmentation_tumor, prediction_tumor)
  return dice_kidney_tumor, dice_tumor

* RVD

In [ ]:
def RVD(mask_auto, manual):  
    tu_p = mask_auto<2
    tk_p= mask_auto>1

    tu_m = manual<2
    tk_m= manual>1

    vol1 = np.count_nonzero(tk_p)
    vol2 = np.count_nonzero(tk_m)  
    if 0 == vol2:
      raise RuntimeError('La maschera manuale non contiene alcun complesso rene-tumore') 
    tk_rvd = (vol1 - vol2) / float(vol2)

    vol1 = np.count_nonzero(tu_p)
    vol2 = np.count_nonzero(tu_m)
    if 0 == vol2:
      raise RuntimeError('La maschera manuale non contiene alcun tumore') 
    tu_rvd = (vol1 - vol2) / float(vol2)

    return tk_rvd, tu_rvd

* Calcolo metriche per il Training

In [ ]:
def calcolo_metriche(mask_auto, mask_manual,original_spacing):
    
    HD_slice = []
    for sl in range(0,len(mask_auto)):
      mask_auto_sl = copy.deepcopy(mask_auto[sl])
      manual = copy.deepcopy(mask_manual[sl])
      # HD va' calcolata sui tumori
      mask_auto_sl[mask_auto_sl<2] = 0
      manual[manual<2] = 0
      index_a = measure.find_contours(mask_auto_sl,level=1)        
      index_m = measure.find_contours(manual,level=1)  
      # La distanza di Hausdorff può essere calcolata nelle slice in cui c'è solo una curva nella mask auto e solo una curva nella mask manual
      if len(index_a) == 1 and len(index_m) == 1:
        contour_a = mask_auto_sl[index_a[0].astype(int)] 
        contour_m = manual[index_m[0].astype(int)] 
        temp_HD = ski_metrics.hausdorff_distance(contour_a, contour_m)
        # calcolo l'HD per ciascuna slice appartenente ad un volume
        HD_slice.append(temp_HD*original_spacing)    # conversione in mm

    DSC_TK,DSC_TU = DSC(mask_manual, mask_auto)
    RVD_TK,RVD_TU = RVD(mask_auto, mask_manual)
    if HD_slice:
      HD_totale = np.sum(np.array(HD_slice))/len(HD_slice)
    else:
      HD_totale = []

    return DSC_TK, DSC_TU, RVD_TK, RVD_TU, HD_totale

### Volume prediction

In [ ]:
def volume_prediction(path_vol, output_path, volumes_info, is_val= False, is_test = False):
  list_vol = sorted(os.listdir(path_vol))
  DSC_TK_vol = []
  DSC_TU_vol = []
  RVD_TK_vol = []
  RVD_TU_vol = []
  HD_vol = []
  for n, id_ in tqdm(enumerate(list_vol), total=len(list_vol)):

    if is_val:
      n = n+100
    if is_test:
      n = n+150
      if n == 160:    # esclusione del caso 160 (dimensioni errate)
        continue
    
    id_ = 'case_' + str(n).zfill(5)

    volume_itk = sitk.ReadImage("/".join((path_vol,id_,'imaging.nii.gz')))
    volume_processed = preprocessing_2D(volume_itk, n, volumes_info)
    volume_processed_2D = np.stack((volume_processed,)*3, axis=-1)

    # PREDICT 2D
    prediction_2D = predict_2D(volume_processed_2D, volumes_info, n)

    # PREPROCESS 3D
    if is_val:
      VOL_1, VOL_2, volumes_info = preprocess_3D(prediction_2D, volume_processed, n, volumes_info, is_val = True)
    elif is_test:
      VOL_1, VOL_2, volumes_info = preprocess_3D(prediction_2D, volume_processed, n, volumes_info, is_test = True)
    else:
      VOL_1, VOL_2, volumes_info = preprocess_3D(prediction_2D, volume_processed, n, volumes_info)

    # PREDICT 3D
    PRED_1 = predict3D(VOL_1)
    PRED_2 = predict3D(VOL_2)

    # RICOSTRUZIONE VOLUME
    prediction_3D = postprocess_3D(PRED_1, PRED_2, n, volumes_info)

    # Sistemiamo la maschera automatica in accordo con l'orientazione dei dati forniti
    prediction_3D = data = np.rot90(prediction_3D, 2, axes=(0, 2))     
    prediction_3D = np.flip(prediction_3D, 2)
    
    # Salvataggio della maschera di segmentazione ottenuta nel formato richiesto
    volume_nifti = load_volume(id_, path_vol)
    header = volume_nifti.header
    spacing = volume_nifti.affine
    # conversione della maschera da numpy array a NIFTI
    nifti = nib.Nifti1Image(prediction_3D, spacing, header)
    # Salva la maschera automatica
    path_save = create_path(output_path,id_)
    nib.save(nifti, os.path.join(path_save,'segmentation_auto.nii.gz'))

In [ ]:
def metriche(path_seg, path_seg_auto, volumes_info, is_val = False):
  list_vol = sorted(os.listdir(path_seg)) 
  DSC_TK_vol = []
  DSC_TU_vol = []
  RVD_TK_vol = []
  RVD_TU_vol = []
  HD_vol = []

  for n, id_ in tqdm(enumerate(list_vol), total=len(list_vol)):
    # if n == 1:
    #   break
    if is_val:
      n = n+100

    segmentation = nib.load("/".join((path_seg,id_,'segmentation.nii.gz')))
    segmentation = segmentation.get_fdata().astype(np.uint8)
    segmentation_auto = nib.load("/".join((path_seg_auto,id_,'segmentation_auto.nii.gz')))
    segmentation_auto = segmentation_auto.get_fdata().astype(np.uint8)
    original_spacing_xy = volumes_info['{}'.format(n)]['original_spacing'][2]
    # CALCOLO METRICHE PER TRAINING E VALIDATION
    DSC_TK, DSC_TU, RVD_TK, RVD_TU, HD = calcolo_metriche(segmentation_auto, segmentation, original_spacing_xy)

    print(DSC_TK)

    DSC_TK_vol.append(DSC_TK) 
    DSC_TU_vol.append(DSC_TU)
    RVD_TK_vol.append(RVD_TK) 
    RVD_TU_vol.append(RVD_TU)
    if HD:
      HD_vol.append(HD)
        
  return DSC_TK_vol, DSC_TU_vol, RVD_TK_vol, RVD_TU_vol, HD_vol

# 7. Salvataggio maschere automatiche

### Training Set

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge/train_info_128'
fp= open(file_path)
volumes_train_info = json.loads(fp.read())

In [ ]:
# Directory da cui prendere i dati
path_train = current_dir+'/'+'training'

# Crea le cartelle di output delle maschere
outpath = create_path(current_dir, 'segmentation_auto')
output_train_path = create_path(outpath, 'training')  

# Predizione e calcolo delle metriche
volume_prediction(path_train, output_train_path, volumes_train_info)
DSC_TK_train_vol, DSC_TU_train_vol, RVD_TK_train_vol, RVD_TU_train_vol, HD_train_vol = metriche(path_train, output_train_path, volumes_train_info)

* Calcolo Metriche Training

In [ ]:
DSC_TK_train = sum(DSC_TK_train_vol)/len(DSC_TK_train_vol)
DSC_TU_train = sum(DSC_TU_train_vol)/len(DSC_TU_train_vol)
RVD_TK_train = sum(RVD_TK_train_vol)/len(RVD_TK_train_vol)
RVD_TU_train = sum(RVD_TU_train_vol)/len(RVD_TU_train_vol)
HD_train = sum(HD_train_vol)/len(HD_train_vol)

print(tabulate([['DSC - rene e tumore', round(DSC_TK_train*100,2), round(np.std(DSC_TK_train_vol)*100)], ['DSC - tumore', round(DSC_TU_train*100,2), round(np.std(DSC_TU_train_vol)*100,2)], ['RVD - rene e tumore', round(RVD_TK_train,2), round(np.std(RVD_TK_train_vol),2)],['RVD - tumore', round(RVD_TU_train,2), round(np.std(RVD_TU_train_vol),2)],['HD  - tumore', round(HD_train,2), round(np.std(HD_train_vol),2)]], headers=["Training Set", "mean", "+/-std"]))

Training Set                 mean       +/-std
-------------------  ------------  -----------
DSC - rene e tumore  95.7541        2.07111
DSC - tumore         76.4161       26.952
RVD - rene e tumore  -0.108559      0.434531
RVD - tumore         -0.000214114   0.00120049
HD  - tumore         19.0676       13.0928


In [ ]:
# Tumour + Kidney
temp = np.asarray(DSC_TK_train_vol)
index_less_95 = np.where(temp<0.95)
index_less_70 = np.where(temp<0.70)
index_less_60 = np.where(temp<0.60)
print(tabulate([['DSC', DSC_TK_train_vol*100], ['RVD_TK_train', RVD_TK_train_vol], ['DSC casi < 95%', *index_less_95], ['DSC casi < 70%', *index_less_70], ['DSC casi < 60%', *index_less_60]], headers=['Training Set', '']))

# Tumour
temp = np.asarray(DSC_TU_train_vol)
index_less_90 = np.where(temp<0.90)
index_less_70 = np.where(temp<0.70)
index_less_60 = np.where(temp<0.60)
#print(tabulate(['DSC', DSC_train*100], ['RVD', RVD_train]], headers=['Training Set', '']))

print(tabulate([['DSC - tumore', DSC_TU_train_vol*100], ['RVD - tumore', RVD_TU_train_vol], ['HD', HD_train_vol], ['DSC casi < 90%', *index_less_90], ['DSC casi < 70%', *index_less_70], ['DSC casi < 60%', *index_less_60]], headers=['Training Set', '']))

Training Set
--------------  -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Validation Set

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge/val_info_128'
fp= open(file_path)
volumes_val_info = json.loads(fp.read())

In [ ]:
# Directory da cui prendere i dati
path_val = current_dir+'/'+'validation'

# Crea le cartelle per i risultati della segmentazione
outpath = create_path(current_dir, 'segmentation_auto')     # CONTROLLO (rimuovere?)
output_val_path = create_path(outpath, 'validation') 

# Predizione e calcolo delle metriche
# DSC_TK_val_vol, DSC_TU_val_vol, RVD_TK_val_vol, RVD_TU_val_vol, HD_val_vol = volume_prediction(path_val, output_val_path, volumes_val_info, is_val = True)
volume_prediction(path_val, output_val_path, volumes_val_info, is_val = True)
DSC_TK_val_vol, DSC_TU_val_vol, RVD_TK_val_vol, RVD_TU_val_vol, HD_val_vol = metriche (path_val, output_val_path, volumes_val_info, is_val = True)

* Calcolo Metriche Validation

In [ ]:
DSC_TK_val = sum(DSC_TK_val_vol)/len(DSC_TK_val_vol)
DSC_TU_val = sum(DSC_TU_val_vol)/len(DSC_TU_val_vol)
RVD_TK_val = sum(RVD_TK_val_vol)/len(RVD_TK_val_vol)
RVD_TU_val = sum(RVD_TU_val_vol)/len(RVD_TU_val_vol)
HD_val = sum(HD_val_vol)/len(HD_val_vol)

print(tabulate([['DSC - rene e tumore', round(DSC_TK_val*100,2), round(np.std(DSC_TK_val_vol)*100)], ['DSC - tumore', round(DSC_TU_val*100,2), round(np.std(DSC_TU_val_vol)*100,2)], ['RVD - rene e tumore', round(RVD_TK_val,2), round(np.std(RVD_TK_val_vol),2)],['RVD - tumore', round(RVD_TU_val,2), round(np.std(RVD_TU_val_vol),2)],['HD  - tumore', round(HD_val,2), round(np.std(HD_train_vol),2)]], headers=["Validation Set", "mean", "+/-std"]))

Validation Set               mean       +/-std
-------------------  ------------  -----------
DSC - rene e tumore  94.4473        3.97369
DSC - tumore         36.0323       33.6463
RVD - rene e tumore   0.263209      5.01039
RVD - tumore         -0.000140114   0.00201065
HD  - tumore         58.2346       33.4705


### Test Set

In [ ]:
 # Directory da cui prendere i dati
path_test = current_dir+'/'+'test'
test_vol = sorted(os.listdir(path_test)) 

# Crea le cartelle di output delle maschere
outpath = create_path(current_dir, 'segmentation_auto')   
output_test_path = create_path(outpath, 'test')

# La funzione volume_prediction viene richiamata solo per l agenerazione delle maschere automatiche; 
# non avendo a disposizione le maschere manuali non è possibile calcolare le metriche per il test set
volume_prediction(path_test, output_test_path, volumes_test_info, is_test=True)